In [1]:
# Importing necessary modules

import sklearn.datasets as datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import tree
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
import pandas as pd 
# read text file into pandas DataFrame and 
# create header 
import os
Column_names = ["TIME_str", "TIME_sec", "latitude", "longitude", "ALTITUDE_FT", "GROUNDTRACK_DEG", \
            "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN", "Vertical_label",\
           "Horizontal_label", "Speed_label", "Turn_rate_deg_per_sec", "REL_HEADING", "DIST_TO_DEST", "MANEUVER_ID" ]
Selected_col_names = [ "ALTITUDE_FT", "GROUNDTRACK_DEG", \
                          "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN",  \
                      "Turn_rate_deg_per_sec", "REL_HEADING", "DIST_TO_DEST"]
Target_category_name = "Horizontal_label"
data_name = "nohold_2"
cwd = os.getcwd()
mypath = cwd + f"/../data/{data_name}/"
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [3]:
count = 0

for file_name in onlyfiles:
    if "Traj" in file_name:
        if count == 0:
            file_path = mypath + file_name
            df = pd.read_csv(file_path, sep=",", header=None)
            df.columns = Column_names
            X_df = df[Selected_col_names]
            Y_target = df[Target_category_name]
        else:
            file_path = mypath + file_name
            df_temp = pd.read_csv(file_path, sep=",", header=None)
            df_temp.columns = Column_names
            
            X_df_temp = df_temp[Selected_col_names]
            Y_target_temp = df_temp[Target_category_name]
            X_df = pd.concat([X_df, X_df_temp])
            Y_target = pd.concat([Y_target, Y_target_temp])
        count +=1
        # if count >= 10:
        #     break
X_df["Turn_rate_deg_per_sec"] = X_df["Turn_rate_deg_per_sec"]*10

In [4]:
data_name = "HOLD_3"
cwd = os.getcwd()
mypath = cwd + f"/../data/{data_name}/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for file_name in onlyfiles:
    if "Traj" in file_name:
        if count == 0:
            file_path = mypath + file_name
            df = pd.read_csv(file_path, sep=",", header=None)
            df.columns = Column_names
            X_df = df[Selected_col_names]
            Y_target = df[Target_category_name]
        else:
            file_path = mypath + file_name
            df_temp = pd.read_csv(file_path, sep=",", header=None)
            df_temp.columns = Column_names
            
            X_df_temp = df_temp[Selected_col_names]
            Y_target_temp = df_temp[Target_category_name]
            X_df = pd.concat([X_df, X_df_temp])
            Y_target = pd.concat([Y_target, Y_target_temp])
        count +=1
        # if count >= 10:
        #     break

In [5]:
target_names =  list(set(list(Y_target)))
word_map_to_num = set(list(Y_target))
print("category:")
word_map_to_num


category:


{' Fly Towards Destination',
 ' Flying opposite established course',
 ' Hold Left Turn -1.5DegPerSec',
 ' Hold Left Turn -3.0DegPerSec',
 ' Hold Right Turn 1.5DegPerSec',
 ' Hold Right Turn 3.0DegPerSec',
 ' Maintain Current Heading',
 ' No Conclusion: Horizontal'}

In [6]:
n_sample = len(X_df)
print("number of points:", n_sample)
X_df = X_df.head(n_sample)
Y_target = Y_target.head(n_sample)
Y = []
for y in Y_target:
    for idx, s in enumerate(word_map_to_num):
        if s == y:
            Y.append(idx)
Y = np.array(Y)

number of points: 190258


In [7]:
import pandas as pd 
import dtreeviz
from sklearn.tree import export_text
# read text file into pandas DataFrame and 
# create header 
import os
cwd = os.getcwd()

# file_name_test = "Traj_AAL80-11121269.dat" # worst
file_name_test = "Traj_AAL245-11147798_L_HOLD.dat" # normal

file_path_test = mypath + file_name_test


# file_name_test = "Traj_AAL1252-11172342POS_HOLD.dat"

# file_path_test = cwd + "/../data/HOLD/" + file_name_test


df_test = pd.read_csv(file_path_test, sep=",", header=None)
df_test.columns = Column_names
X_df_test = df_test[Selected_col_names]

In [8]:
print("number of testing points:", len(X_df_test))

number of testing points: 3407


In [9]:
Z_target = df_test[Target_category_name]
n_sample = len(X_df_test)
X_df_test = X_df_test.head(n_sample)
Z_target = Z_target.head(n_sample)
Z = []
for z in Z_target:
    for idx, s in enumerate(word_map_to_num):
        if s == z:
            Z.append(idx)
Z = np.array(Z)
len(Z)

3407

## target 1: Horizontal_label

# Entropy

In [10]:
data_name = "nohold_2_OR_HOLD_3"

In [11]:
# Entropy, small tree with train data

criterion = "entropy"
max_depth = 6
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
viz_model = dtreeviz.model(dtree_small_entropy, X_df, Y,
                target_name="target",
                feature_names=list(X_df.columns),
                class_names=target_names)

v = viz_model.view()     # render as SVG into internal object 
v                 # pop up window
v.save(f"{data_name}_all_{Target_category_name}_{criterion}_without_xy_" + f"max_depth:{max_depth}"  + ".svg")  # optionally save as svg

0.7782169475133766


/Users/binshuaiwang/PycharmProjects/XAI_project/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names


In [12]:
# Entropy, small tree with train data
from sklearn.tree import export_text
from sklearn.tree import export_text_in_conditions

criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
viz_model = dtreeviz.model(dtree_small_entropy, X_df, Y,
                target_name="target",
                feature_names=list(X_df.columns),
                class_names=target_names)

v = viz_model.view()     # render as SVG into internal object 
v                 # pop up window
v.save(f"{data_name}_all_{Target_category_name}_{criterion}_without_xy_" + f"max_depth:{max_depth}"  + ".svg")  # optionally save as svg
r = export_text_in_conditions(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)



0.7782169475133766


/Users/binshuaiwang/PycharmProjects/XAI_project/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names


If ALTITUDE_FT <= 4973.50 && REL_HEADING <= -119.88 && GROUNDSPEED_KTS <= 0.50 && DIST_TO_DEST <= 180.37, then  class:  Flying opposite established course

If ALTITUDE_FT <= 4973.50 && REL_HEADING <= -119.88 && GROUNDSPEED_KTS <= 0.50 && DIST_TO_DEST > 180.37, then  class:  Flying opposite established course

If ALTITUDE_FT <= 4973.50 && REL_HEADING <= -119.88 && GROUNDSPEED_KTS > 0.50 && REL_HEADING <= -141.17 && ALTITUDE_FT <= 255.50 && REL_HEADING <= -160.79, then  class:  Flying opposite established course

If ALTITUDE_FT <= 4973.50 && REL_HEADING <= -119.88 && GROUNDSPEED_KTS > 0.50 && REL_HEADING <= -141.17 && ALTITUDE_FT <= 255.50 && REL_HEADING > -160.79, then  class:  Flying opposite established course

If ALTITUDE_FT <= 4973.50 && REL_HEADING <= -119.88 && GROUNDSPEED_KTS > 0.50 && REL_HEADING <= -141.17 && ALTITUDE_FT > 255.50 && Turn_rate_deg_per_sec <= 6.50, then  class:  Flying opposite established course

If ALTITUDE_FT <= 4973.50 && REL_HEADING <= -119.88 && GROUNDSPEED

In [13]:
import graphviz
from sklearn.tree import export_graphviz
dot_data = export_graphviz(dtree_small_entropy,
                           feature_names=X_df.columns,  
                           class_names = target_names,

                           filled=True,  
                           rounded = True,
                            precision = 2,
                           proportion=True)
graph = graphviz.Source(dot_data)
graph.format = "png"
graph.render(f"{data_name}__all_{Target_category_name}_{criterion}" + "importances")

'nohold_2_OR_HOLD_3__all_Horizontal_label_entropyimportances.png'

In [14]:
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)

In [15]:
print(r)

|--- ALTITUDE_FT <= 4973.50
|   |--- REL_HEADING <= -119.88
|   |   |--- GROUNDSPEED_KTS <= 0.50
|   |   |   |--- DIST_TO_DEST <= 180.37
|   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- DIST_TO_DEST >  180.37
|   |   |   |   |--- class:  Flying opposite established course
|   |   |--- GROUNDSPEED_KTS >  0.50
|   |   |   |--- REL_HEADING <= -141.17
|   |   |   |   |--- ALTITUDE_FT <= 255.50
|   |   |   |   |   |--- REL_HEADING <= -160.79
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- REL_HEADING >  -160.79
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- ALTITUDE_FT >  255.50
|   |   |   |   |   |--- Turn_rate_deg_per_sec <= 6.50
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- Turn_rate_deg_per_sec >  6.50
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -141.1

In [16]:
tree = model.tree_

In [17]:
print(tree.children_left[tree.children_left[2]])

4


In [18]:
criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=10, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.8491784839533686
|--- ALTITUDE_FT <= 4973.50
|   |--- REL_HEADING <= -119.88
|   |   |--- GROUNDSPEED_KTS <= 0.50
|   |   |   |--- DIST_TO_DEST <= 180.37
|   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- DIST_TO_DEST >  180.37
|   |   |   |   |--- class:  Flying opposite established course
|   |   |--- GROUNDSPEED_KTS >  0.50
|   |   |   |--- REL_HEADING <= -141.17
|   |   |   |   |--- ALTITUDE_FT <= 255.50
|   |   |   |   |   |--- REL_HEADING <= -160.79
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 13.50
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  13.50
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- REL_HEADING >  -160.79
|   |   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -96.00
|   |   |   |   |   |   |   |--- DIST_TO_DEST <= 196.20
|   |   |   |   |   |   |   |   |--- DIST_TO_DEST <= 195.91
|   |   |   |   |   

In [19]:
# Entropy, large tree with train data

criterion = "entropy"
dtree_large_n = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_n.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_n, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9982287209999055
|--- ALTITUDE_FT <= 4973.50
|   |--- REL_HEADING <= -119.88
|   |   |--- GROUNDSPEED_KTS <= 0.50
|   |   |   |--- DIST_TO_DEST <= 180.37
|   |   |   |   |--- DIST_TO_DEST <= 180.37
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- DIST_TO_DEST >  180.37
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- DIST_TO_DEST >  180.37
|   |   |   |   |--- class:  Flying opposite established course
|   |   |--- GROUNDSPEED_KTS >  0.50
|   |   |   |--- REL_HEADING <= -141.17
|   |   |   |   |--- ALTITUDE_FT <= 255.50
|   |   |   |   |   |--- REL_HEADING <= -160.79
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 13.50
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  13.50
|   |   |   |   |   |   |   |--- DIST_TO_DEST <= 195.89
|   |   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |   

In [20]:
# Entropy, large tree with test data

criterion = "entropy"
dtree_large_entropy = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9999999999999999
|--- ALTITUDE_FT <= 4973.50
|   |--- REL_HEADING <= -119.88
|   |   |--- GROUNDSPEED_KTS <= 0.50
|   |   |   |--- DIST_TO_DEST <= 180.37
|   |   |   |   |--- DIST_TO_DEST <= 180.37
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- DIST_TO_DEST >  180.37
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- DIST_TO_DEST >  180.37
|   |   |   |   |--- class:  Flying opposite established course
|   |   |--- GROUNDSPEED_KTS >  0.50
|   |   |   |--- REL_HEADING <= -141.17
|   |   |   |   |--- ALTITUDE_FT <= 255.50
|   |   |   |   |   |--- REL_HEADING <= -160.79
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 13.50
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  13.50
|   |   |   |   |   |   |   |--- DIST_TO_DEST <= 195.89
|   |   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |   

In [21]:
# Entropy, small tree with test data

criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=10, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.7713530965658937
|--- ALTITUDE_FT <= 4973.50
|   |--- REL_HEADING <= -119.88
|   |   |--- GROUNDSPEED_KTS <= 0.50
|   |   |   |--- DIST_TO_DEST <= 180.37
|   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- DIST_TO_DEST >  180.37
|   |   |   |   |--- class:  Flying opposite established course
|   |   |--- GROUNDSPEED_KTS >  0.50
|   |   |   |--- REL_HEADING <= -141.17
|   |   |   |   |--- ALTITUDE_FT <= 255.50
|   |   |   |   |   |--- REL_HEADING <= -160.79
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 13.50
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  13.50
|   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- REL_HEADING >  -160.79
|   |   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -96.00
|   |   |   |   |   |   |   |--- DIST_TO_DEST <= 196.20
|   |   |   |   |   |   |   |   |--- DIST_TO_DEST <= 195.91
|   |   |   |   |   

In [22]:
# Entropy, large rf with test data

criterion = "entropy"
num_of_trees = 3
rf_small_entropy = RandomForestClassifier(n_estimators=num_of_trees)
model=rf_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9932491928382741


In [23]:
# Entropy, small rf with test data

criterion = "entropy"
num_of_trees = 3
rf_small_entropy = RandomForestClassifier(n_estimators=num_of_trees, max_depth=5, min_samples_leaf=20)
model=rf_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.5964191370707367


# Gini

In [24]:
# Gini, small tree with train data

criterion = "gini"
dtree_small_gini = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_gini.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.7624961893849405
|--- ALTITUDE_FT <= 4997.50
|   |--- REL_HEADING <= -120.15
|   |   |--- Turn_rate_deg_per_sec <= -0.50
|   |   |   |--- REL_HEADING <= -142.17
|   |   |   |   |--- REL_HEADING <= -160.79
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- REL_HEADING >  -160.79
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -142.17
|   |   |   |   |--- ALTITUDE_FT <= 258.50
|   |   |   |   |   |--- class:  Hold Right Turn 1.5DegPerSec
|   |   |   |   |--- ALTITUDE_FT >  258.50
|   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |--- Turn_rate_deg_per_sec >  -0.50
|   |   |   |--- VERTICALSPEED_FT_PER_MIN <= 1632.00
|   |   |   |   |--- REL_HEADING <= -122.15
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- REL_HEADING >  -122.15
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- VERTICALSPEED_FT_PER_MIN 

In [25]:
# Gini, large tree with train data
criterion = "gini"
dtree_large_gini = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_gini.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9982287209999055
|--- ALTITUDE_FT <= 4997.50
|   |--- REL_HEADING <= -120.15
|   |   |--- Turn_rate_deg_per_sec <= -0.50
|   |   |   |--- REL_HEADING <= -142.17
|   |   |   |   |--- GROUNDSPEED_KTS <= 2.50
|   |   |   |   |   |--- DIST_TO_DEST <= 196.27
|   |   |   |   |   |   |--- class:  Hold Right Turn 1.5DegPerSec
|   |   |   |   |   |--- DIST_TO_DEST >  196.27
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- GROUNDSPEED_KTS >  2.50
|   |   |   |   |   |--- Turn_rate_deg_per_sec <= -770.00
|   |   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |   |--- Turn_rate_deg_per_sec >  -770.00
|   |   |   |   |   |   |--- DIST_TO_DEST <= 345.48
|   |   |   |   |   |   |   |--- REL_HEADING <= -151.92
|   |   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |   |--- REL_HEADING >  -151.92
|   |   |   |   |   |   |   |   |--- GROUNDTRACK_DEG <= 78.50
|   |   |   |   |   |   |   

In [26]:
# Gini, large tree with test data

criterion = "gini"
dtree_large_gini = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_gini.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9999999999999999
|--- ALTITUDE_FT <= 4997.50
|   |--- REL_HEADING <= -120.15
|   |   |--- Turn_rate_deg_per_sec <= -0.50
|   |   |   |--- REL_HEADING <= -142.17
|   |   |   |   |--- GROUNDSPEED_KTS <= 2.50
|   |   |   |   |   |--- GROUNDTRACK_DEG <= 172.50
|   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |--- GROUNDTRACK_DEG >  172.50
|   |   |   |   |   |   |--- class:  Hold Right Turn 1.5DegPerSec
|   |   |   |   |--- GROUNDSPEED_KTS >  2.50
|   |   |   |   |   |--- Turn_rate_deg_per_sec <= -770.00
|   |   |   |   |   |   |--- class:  No Conclusion: Horizontal
|   |   |   |   |   |--- Turn_rate_deg_per_sec >  -770.00
|   |   |   |   |   |   |--- DIST_TO_DEST <= 345.48
|   |   |   |   |   |   |   |--- REL_HEADING <= -151.92
|   |   |   |   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |   |   |   |--- REL_HEADING >  -151.92
|   |   |   |   |   |   |   |   |--- DIST_TO_DEST <= 300.59
|   |   |   |   |   |   

In [27]:
# Gini, small tree with test data

criterion = "gini"
dtree_small_gini = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_gini.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.563839154681538
|--- ALTITUDE_FT <= 4997.50
|   |--- REL_HEADING <= -120.15
|   |   |--- Turn_rate_deg_per_sec <= -0.50
|   |   |   |--- REL_HEADING <= -142.17
|   |   |   |   |--- REL_HEADING <= -160.79
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- REL_HEADING >  -160.79
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -142.17
|   |   |   |   |--- ALTITUDE_FT <= 258.50
|   |   |   |   |   |--- class:  Hold Right Turn 1.5DegPerSec
|   |   |   |   |--- ALTITUDE_FT >  258.50
|   |   |   |   |   |--- class:  Hold Left Turn -1.5DegPerSec
|   |   |--- Turn_rate_deg_per_sec >  -0.50
|   |   |   |--- VERTICALSPEED_FT_PER_MIN <= 1632.00
|   |   |   |   |--- REL_HEADING <= -122.15
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- REL_HEADING >  -122.15
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- VERTICALSPEED_FT_PER_MIN >

## Log_loss

In [28]:
# Log_loss, small tree with train data

criterion = "log_loss"
dtree_small_Log_loss = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_Log_loss, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.7616079218745072
|--- ALTITUDE_FT <= 4973.50
|   |--- REL_HEADING <= -119.88
|   |   |--- GROUNDSPEED_KTS <= 0.50
|   |   |   |--- DIST_TO_DEST <= 180.37
|   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- DIST_TO_DEST >  180.37
|   |   |   |   |--- class:  Flying opposite established course
|   |   |--- GROUNDSPEED_KTS >  0.50
|   |   |   |--- REL_HEADING <= -141.17
|   |   |   |   |--- ALTITUDE_FT <= 255.50
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- ALTITUDE_FT >  255.50
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |--- REL_HEADING >  -141.17
|   |   |   |   |--- GROUNDTRACK_DEG <= 266.50
|   |   |   |   |   |--- class:  Flying opposite established course
|   |   |   |   |--- GROUNDTRACK_DEG >  266.50
|   |   |   |   |   |--- class:  Flying opposite established course
|   |--- REL_HEADING >  -119.88
|   |   |--- REL_HEADING <= 120.03
|   |   |   |--- GROUNDSPEED_KTS <= 0.50

In [29]:
# Log_loss, large tree with train data
criterion = "log_loss"
dtree_large_Log_loss = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9982287209999055


In [30]:
# Log_loss, large tree with test data
criterion = "log_loss"
dtree_large_Log_loss = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9999999999999999


In [31]:
# Log_loss, small tree with test data

criterion = "log_loss"
dtree_small_Log_loss = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.5600234810683886
